In [19]:
library(Seurat)
library(tidyverse)
library(ggplot2)
library(patchwork)
library(forcats)
library(Polychrome)
library(grid)

In [ ]:
xenium.obj = qs::qread('_targets/objects/obj_merged')
# xenium.obj = qs::qread('../01_resolve/230710_xenium_sct_unimapped_cca_polar_label.qs')
merged_meta = qs::qread('_targets/objects/obj_rctd_merged_01') %>% `[[`
selected_cells = merged_meta %>% rownames
xenium.obj = xenium.obj %>% subset(cells = selected_cells) %>% AddMetaData(merged_meta)
xenium.obj


In [45]:
a = qs::qread('_targets/objects/obj_rctd_merged_01_sv4')

In [47]:
qs::qread('_targets/objects/obj_rctd_merged_01_sv4')@images$fov@boundaries$segmentation@polygons$Cell1003_6744_1600_1

An object of class "Polygons"
Slot "Polygons":
[[1]]
An object of class "Polygon"
Slot "labpt":
[1] 6743.868 1600.696

Slot "area":
[1] 1115

Slot "hole":
[1] FALSE

Slot "ringDir":
[1] 1

Slot "coords":
       x    y
457 6739 1584
458 6734 1586
459 6723 1600
460 6721 1603
461 6717 1612
462 6730 1614
463 6758 1613
464 6766 1609
465 6767 1608
466 6769 1602
467 6768 1599
468 6753 1585
469 6751 1584
470 6739 1584



Slot "plotOrder":
[1] 1

Slot "labpt":
[1] 6743.868 1600.696

Slot "ID":
[1] "Cell1003_6744_1600_1"

Slot "area":
[1] 1115


In [48]:
qs::qread('_targets/objects/obj_rctd_merged_01')@images$fov@boundaries$segmentation@polygons$Cell1003_6744_1600_1

An object of class "Polygons"
Slot "Polygons":
[[1]]
An object of class "Polygon"
Slot "labpt":
[1] 6743.868 1600.696

Slot "area":
[1] 1115

Slot "hole":
[1] FALSE

Slot "ringDir":
[1] 1

Slot "coords":
       x    y
457 6739 1584
458 6734 1586
459 6723 1600
460 6721 1603
461 6717 1612
462 6730 1614
463 6758 1613
464 6766 1609
465 6767 1608
466 6769 1602
467 6768 1599
468 6753 1585
469 6751 1584
470 6739 1584



Slot "plotOrder":
[1] 1

Slot "labpt":
[1] 6743.868 1600.696

Slot "ID":
[1] "Cell1003_6744_1600_1"

Slot "area":
[1] 1115


In [ ]:
xenium.obj@meta.data = xenium.obj@meta.data %>%
mutate(predicted.polar_label = polar_label)

In [ ]:
rownames(xenium.obj)

In [ ]:
create_grouping_col <- function(seurat_obj, prefix) {
  new_metadata <- case_when(str_detect(seurat_obj@meta.data$predicted.polar_label, prefix) ~ seurat_obj@meta.data$predicted.polar_label,
                            TRUE ~ NA_character_) %>% as.factor
  new_metadata = new_metadata %>% fct_relevel(sort(na.omit(levels(new_metadata))), after = Inf)
  return(new_metadata)
}



In [ ]:
plot_celltype_idp = function(seurat_obj, cell_str, fov='fov'){
    new_grouping = seurat_obj %>%
        create_grouping_col(cell_str)
    seurat_obj = AddMetaData(seurat_obj, new_grouping, col.name = 'grouping_col')

    # Get the levels of the lab_Pomc column, including the NA_character_ level
    grouping_levels <- levels(seurat_obj@meta.data$grouping_col)
    # Generate colors using the polychrome palette
    how_many_colors = length(grouping_levels)
    colors <- Polychrome::sky.colors(how_many_colors) %>% as.character %>% `[`(1:how_many_colors) # Excluding the NA_character_ level
    # Add 'gray10' for the NA_character_ level
    colors <- c(colors, "gray10")
    # Create a named vector of colors, with levels as names
    named_colors <- setNames(colors, grouping_levels)
    # Map the lab_Pomc column values to the corresponding colors
    # color_vector <- mapvalues(xenium.obj@meta.data$lab_Pomc, from = lab_Pomc_levels, to = named_colors)
    
    ggp = ImageDimPlot(seurat_obj,
                       group.by='grouping_col',
                       boundaries = 'segmentation',
                       fov = fov,
                       border.size =0.1, 
                       border.color =NA,
                       na.value='gray20', 
                       cols=named_colors, 
                       axes = FALSE,
                       size=0)
    ggp
}

In [ ]:
make_idp_pdf = function(seurat_obj, cell_type, prefix="^", suffix="\\.(neg|none|pos)", pdf_folder=""){
    if (!file.exists(pdf_folder)) {
      dir.create(pdf_folder, showWarnings = F)
    }
    cell_type_regex = cell_type %>% paste0(prefix, ., suffix)
    dir.create(pdf_folder, recursive = TRUE, showWarnings = FALSE)
    a1 = plot_celltype_idp(seurat_obj, cell_type_regex, fov='fov')  + theme(legend.position = "none")
    a2 = plot_celltype_idp(seurat_obj, cell_type_regex, fov='fov.2') + theme(legend.position = "none")
    b1 = plot_celltype_idp(seurat_obj, cell_type_regex, fov='fov.3') + theme(legend.position = "none")
    b2 = plot_celltype_idp(seurat_obj, cell_type_regex, fov='fov.4') + theme(legend.position = "none")
    c1 = plot_celltype_idp(seurat_obj, cell_type_regex, fov='fov.5') + theme(legend.position = "none")
    c2 = plot_celltype_idp(seurat_obj, cell_type_regex, fov='fov.6') + theme(legend.position = "none")
    d1 = plot_celltype_idp(seurat_obj, cell_type_regex, fov='fov.7') + theme(legend.position = "none")
    d2 = plot_celltype_idp(seurat_obj, cell_type_regex, fov='fov.8') + theme(legend.position = "none")
    title_theme <- theme(plot.title = element_text(color = "white", size = 8, hjust = 0.5))
    # Set black background theme for each plot
    legend_theme <- theme(legend.position = "bottom",
                          legend.direction = "horizontal",
                          legend.box = "horizontal",
                          legend.text = element_text(size = 4),
                          legend.key.size = unit(2, "mm"))
    # black_background_theme <- theme(plot.background = element_rect(fill = "black"),
    #                                 panel.background = element_rect(fill = "black"))
    black_background_theme <- theme(plot.background = element_rect(fill = "black", color = 'black'),
                          panel.background = element_rect(fill = "black", color = 'black'),
                          plot.margin = margin(0, 0, 0, 0),
                          panel.grid.major = element_blank(),
                          panel.grid.minor = element_blank(),
                          panel.border = element_blank())

    a1_with_title <- a1 + theme(legend.position = "right") + ggtitle("a1_Veh_PF") + title_theme + legend_theme + black_background_theme
    a2_no_legend_with_title <- a2 + guides(scale="none") + ggtitle("a2_Veh_PF") + title_theme + black_background_theme
    b1_no_legend_with_title <- b1 + guides(scale="none") + ggtitle("b1_Veh_PF") + title_theme + black_background_theme
    b2_no_legend_with_title <- b2 + guides(scale="none") + ggtitle("b2_Veh_PF") + title_theme + black_background_theme
    c1_no_legend_with_title <- c1 + guides(scale="none") + ggtitle("c1_FGF1") + title_theme + black_background_theme
    c2_no_legend_with_title <- c2 + guides(scale="none") + ggtitle("c2_FGF1") + title_theme + black_background_theme
    d1_no_legend_with_title <- d1 + guides(scale="none") + ggtitle("d1_FGF1") + title_theme + black_background_theme
    d2_no_legend_with_title <- d2 + guides(scale="none") + ggtitle("d2_FGF1") + title_theme + black_background_theme

    # Combine the plots using patchwork with a black background
    combined_plots <- a1_with_title + c1_no_legend_with_title +
                      a2_no_legend_with_title + c2_no_legend_with_title +
                      b1_no_legend_with_title + d1_no_legend_with_title +
                      b2_no_legend_with_title + d2_no_legend_with_title +
                      plot_layout(guides = "collect", ncol = 2, heights = c(1, 1, 1, 1), widths = c(1, 1))
    
    # combined_plots = combined_plots & black_background_theme & legend_theme 
    combined_plots = combined_plots + plot_annotation(title=cell_type, theme=black_background_theme) + plot_annotation(theme=legend_theme) + plot_annotation(theme=title_theme)
    # Save the combined_plots to a PDF with a black background
    pdf_name = paste0(pdf_folder, cell_type, '_ct_mapping.pdf')
    pdf(pdf_name, width = 8.27, height = 11.69, bg = "black")
    # grid.newpage()
    grid.draw(combined_plots)
    dev.off()
}

In [ ]:
cell_types = xenium.obj %>% `[[` %>% 
group_by(predicted.polar_label) %>% 
summarise(n = n()) %>% 
arrange(desc(n)) %>% 
pull(predicted.polar_label) %>% 
str_replace(fixed('.none'), '') %>%
str_replace(fixed('.pos'), '') %>%
str_replace(fixed('.neg'), '') %>%
unique
cell_types

In [ ]:
make_idp_pdf(xenium.obj, "OPC", prefix="", suffix="", pdf_folder="ct_mapping_rctd_polar_label/")

In [ ]:
for (cell_type in cell_types){
    print(cell_type)
    make_idp_pdf(xenium.obj, cell_type, pdf_folder="ct_mapping_rctd_polar_label/")
}

In [41]:
xenium.obj@images$fov@boundaries$segmentation@polygons$Cell1_4012_26_1

An object of class "Polygons"
Slot "Polygons":
[[1]]
An object of class "Polygon"
Slot "labpt":
[1] 4012.76072   25.60736

Slot "area":
[1] 2768

Slot "hole":
[1] FALSE

Slot "ringDir":
[1] 1

Slot "coords":
         x  y
85656 3994  2
85657 3988  3
85658 3981 11
85659 3986 42
85660 3988 47
85661 3997 54
85662 4015 57
85663 4022 50
85664 4048 22
85665 4054 12
85666 4053  9
85667 4040  6
85668 4023  3
85669 3997  2
85670 3994  2



Slot "plotOrder":
[1] 1

Slot "labpt":
[1] 4012.76072   25.60736

Slot "ID":
[1] "Cell1_4012_26_1"

Slot "area":
[1] 2768


In [ ]:
xe